In [ ]:
def speckle_filter(data_ma, kernel):
    
    mask = data_ma.mask.copy()
    
    # Label outliers by proximity
    """
    Returns an array with gates indexed according to the group they belong to (labs) 
    and the number of groups found (n_labs)
    """
    labs, n_labs = sp.ndimage.label((~mask))
     
    # Number of gates in each group
    bincount = np.bincount(labs.ravel())
    region_sizes = bincount[1:]
    
    return region_sizes